In [1]:
!pip install datasets transformers sentence-transformers faiss-cpu tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import json
import pickle
import numpy as np
import torch
from tqdm import tqdm
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss
from google.colab import files  # Colab에서만 필요, 로컬에서는 제거

# GPU 사용 가능 여부 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"사용 장치: {device}")

사용 장치: cuda


#### Dataset Load

In [3]:
dataset = load_dataset("squad_kor_v1")
print(f"데이터셋 로드 완료: {len(dataset['train'])} 학습 샘플, {len(dataset['validation'])} 검증 샘플")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.29k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

데이터셋 로드 완료: 60407 학습 샘플, 5774 검증 샘플


#### Document 전처리

In [4]:
documents = []

# 학습 데이터와 검증 데이터 모두 활용
for split in ['train', 'validation']:
    for item in tqdm(dataset[split]):
        doc = {
            'id': item['id'],
            'title': item['title'],
            'context': item['context'],
            'question': item['question'],
            'answers': item['answers']
        }
        documents.append(doc)

# 중복 제거 (context 기준)
unique_contexts = {}
unique_docs = []

for doc in documents:
    if doc['context'] not in unique_contexts:
        unique_contexts[doc['context']] = doc
        unique_docs.append(doc)

print(f"전처리된 문서 수: {len(documents)} (중복 제거 후: {len(unique_docs)})")

# 중복 제거 후의 문서 목록 사용
documents = unique_docs

100%|██████████| 5774/5774 [00:00<00:00, 9713.77it/s]


전처리된 문서 수: 66181 (중복 제거 후: 10563)


#### 임베딩 모델 로딩
- 임베딩 모델 선택 벤치마크 참고
- https://huggingface.co/dragonkue/snowflake-arctic-embed-l-v2.0-ko

아래의 두 임베딩 모델은 GPU 램 이슈로 실행 실패.. 코랩 결제 불가 이슈로 인한
- dragonkue/snowflake-arctic-embed-l-v2.0-ko
- dragonkue/BGE-m3-ko

아래의 모델은 성공
- nlpai-lab/KURE-v

In [5]:
model_name = 'nlpai-lab/KURE-v1'
model = SentenceTransformer(model_name, device=device)
print(f"임베딩 모델 '{model_name}' 로드 완료 (임베딩 차원: {model.get_sentence_embedding_dimension()})")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/807 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

임베딩 모델 'nlpai-lab/KURE-v1' 로드 완료 (임베딩 차원: 1024)


#### 임베딩 생성 및 FAISS 인덱스 구축
- 고차원 벡터 공간에서 빠른 유사도 검색을 위해 특별히 설계
- 임베딩과 같은 고차원 벡터(예: 768차원, 1024차원)에서 최근접 이웃(Nearest Neighbors)을 찾는 작업에 최적화

* 이러한 이유때문에 FAISS 사용

In [6]:
# context에 대한 임베딩 생성
contexts = [doc['context'] for doc in documents]
embeddings = []

# 배치 처리로 메모리 효율성 향상
batch_size = 16
for i in tqdm(range(0, len(contexts), batch_size)):
    batch = contexts[i:i + batch_size]
    with torch.no_grad():
        batch_embeddings = model.encode(batch, convert_to_tensor=True)
        embeddings.extend(batch_embeddings.cpu().numpy())

embeddings = np.array(embeddings).astype('float32')
print(f"임베딩 생성 완료: {embeddings.shape}")

# FAISS 인덱스 생성
dimension = embeddings.shape[1]

# IVF 인덱스 (더 빠른 검색을 위한 옵션)
nlist = min(4096, max(int(np.sqrt(len(documents))), 50))  # 클러스터 수 설정
quantizer = faiss.IndexFlatIP(dimension)
index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_INNER_PRODUCT)

# 인덱스 학습 (IVF의 경우 필요)
if not index.is_trained:
    print(f"FAISS 인덱스 학습 중 (nlist={nlist})...")
    index.train(embeddings)

# 인덱스에 벡터 추가
print("인덱스에 벡터 추가 중...")
index.add(embeddings)
print(f"FAISS 인덱스 생성 완료: {index.ntotal} 벡터")

100%|██████████| 661/661 [17:31<00:00,  1.59s/it]


임베딩 생성 완료: (10563, 1024)
FAISS 인덱스 학습 중 (nlist=102)...
인덱스에 벡터 추가 중...
FAISS 인덱스 생성 완료: 10563 벡터


#### 데이터 저장

In [7]:
# 데이터 저장 디렉토리 생성
os.makedirs('korquad_data', exist_ok=True)

# FAISS 인덱스 저장
print("FAISS 인덱스 저장 중...")
faiss.write_index(index, 'korquad_data/document_index.faiss')

# 문서 메타데이터 저장
print("문서 메타데이터 저장 중...")
with open('korquad_data/documents.json', 'w', encoding='utf-8') as f:
    json.dump(documents, f, ensure_ascii=False, indent=2)

# ID와 인덱스 매핑 저장 (검색 결과 매핑용)
id_to_index = {doc['id']: i for i, doc in enumerate(documents)}
with open('korquad_data/id_to_index.pkl', 'wb') as f:
    pickle.dump(id_to_index, f)

print("데이터 저장 완료")

FAISS 인덱스 저장 중...
문서 메타데이터 저장 중...
데이터 저장 완료


#### 다운로드

In [10]:
!zip -r korquad_data.zip korquad_data/

files.download('korquad_data.zip')

updating: korquad_data/ (stored 0%)
updating: korquad_data/documents.json (deflated 68%)
updating: korquad_data/document_index.faiss (deflated 7%)
updating: korquad_data/id_to_index.pkl (deflated 74%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>